## Import Libs

In [25]:
#import basic Libraries
import pandas as pd
import numpy as np
import numpy_indexed as npi
from tqdm import tqdm

#Visualizaiton imports
import seaborn as sns
import matplotlib.pyplot as plt

#combine sparse matrices
import scipy.sparse as sps

#Importing Sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import MiniBatchDictionaryLearning
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error,f1_score 
    
%matplotlib inline

In [2]:
filenames = ['aisles','departments' ,'order_products__prior','order_products__train', 'orders','products']

In [3]:
#import of all files from the data folder.
aisles = pd.read_csv('./data/'+ filenames[0] + '.csv')
departments = pd.read_csv('./data/'+ filenames[1] + '.csv')
order_products__prior = pd.read_csv('./data/'+ filenames[2] + '.csv')
order_products__train = pd.read_csv('./data/'+ filenames[3] + '.csv')
orders = pd.read_csv('./data/'+ filenames[4] + '.csv')
products = pd.read_csv('./data/'+ filenames[5] + '.csv')

In [4]:
opt = order_products__train.merge(products,how='left', on='product_id')
opt = opt.merge(departments,how='left', on='department_id')
opt = opt.merge(aisles,how='left', on='aisle_id')

## Setting up Logistic Regression to Classifiy reorders

In [5]:
X = opt.drop(columns=['reordered','product_name','department','aisle'])
y = opt['reordered']

In [6]:
X.shape

(1384617, 5)

In [7]:
opt.shape

(1384617, 9)

 ## Train Test Split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [9]:
lr = LogisticRegression()

In [10]:
lr.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [11]:
predictions = lr.predict(X)

In [12]:
lr.score(X_test,y_test)

0.5983855979739326

## Testing on the prior data set

In [13]:
print(order_products__prior.drop('reordered',axis=1).shape)
print(order_products__prior['reordered'].shape)

(32434489, 3)
(32434489,)


In [14]:
lr.fit(order_products__prior.drop('reordered',axis=1),order_products__prior['reordered'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [15]:
lr.score(order_products__prior.drop('reordered',axis=1),order_products__prior['reordered'])

0.5896974667922161

In [23]:
lr.predict_proba(order_products__prior.drop('reordered',axis=1))

array([[0.44461816, 0.55538184],
       [0.45148596, 0.54851404],
       [0.48434463, 0.51565537],
       ...,
       [0.42099563, 0.57900437],
       [0.38824302, 0.61175698],
       [0.42030819, 0.57969181]])

In [24]:
lr.coef_

array([[ 8.41234163e-08,  6.71617939e-06, -5.44139146e-09]])

In [33]:
y_test.shape
#y_train.shape

(415386,)

In [42]:
predictions

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [47]:
f1_score(y,predictions)

0.7489009194281664